# Azimuth oscillation analysis

Analyzing the go-and-come-back idea
Craig Lage - 26Sep23

In [ ]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(5,1,figsize = (8,12))
plt.subplots_adjust(hspace=0.5)
images = [[6, 96, "2023-09-22", "Baseline", 52], [7, 97, "2023-09-26", "Go-and-come-back: 2 degree overshoot, 2 sec wait", 65], \
         [98, 188, "2023-09-26", "Go-and-come-back: 2 degree overshoot, 0 sec wait", 62], \
         [189, 245, "2023-09-26", "Go-and-come-back: 1 degree overshoot, 0 sec wait", 61], \
         [15, 92, "2023-09-26", "Programmed overslew", 0]]
for i, [firstImage, lastImage, dayObs, name, cycleTime] in enumerate(images):
    # Get Rubin TV JSON
    filename = f'/scratch/cslage/starTracker/auxtel_{dayObs}.json'
    print(filename)
    df = pd.read_json(filename)
    df = df.transpose()
    start = Time(f"{dayObs}T{df.loc[firstImage]['TAI']}", format='isot', scale='tai')
    end = Time(f"{dayObs}T{df.loc[lastImage]['TAI']}", format='isot', scale='tai')
    az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", "azimuthCalculatedAngle",\
                                                start.utc, end.utc)
    azs = az['azimuthCalculatedAngle'].values
    times = az['times'].values
    t0 = times[0]
    times -= t0
    tmax = times[-1]

    axs[i].set_title(f"{name} - {dayObs}", fontsize=16)
    axs[i].plot(times, azs)
    axs[i].set_ylabel("Azimuth(degrees)")
    axs[i].set_xlabel("Time(seconds)")
    axs[i].set_xlim(0,6100)
    #axs[i].text(4500, 200, f"Cycle time = {cycleTime} sec.")
    ax2 = axs[i].twinx()
    mount_times = []
    mount_motions = []
    for seqNum in df.index.values.tolist():
        if seqNum >= firstImage and seqNum <= lastImage:
            time = Time(f"{dayObs}T{df.loc[seqNum]['TAI']}", format='isot', scale='tai')
            mount_times.append(time.utc.unix_tai - t0)
            mount_motions.append(df.loc[seqNum]['Mount motion image degradation'])
    ax2.scatter(mount_times, mount_motions, marker='x', color='g')
    ax2.set_ylabel("Mount jitter (arcseconds)")
    ax2.set_ylim(0, 1.5)
#plt.savefig(f"/home/craiglagegit/DATA/Go-and-Come-Back_Test_26Sep23.png")
